## Transform ResStock Run Results to 15 SF prototypes (Elevate Energy)

In [3]:
import pandas as pd
import numpy as np
import os
print(f'Notebook path: {os.getcwd()}')

Notebook path: /Users/lliu2/Documents/GitHub/OpenStudio-BuildStock/files


### Download results from S3:
://S3/resbldg-datasets/chicagoeui

In [4]:
# local path to downloaded results
iteration = '_08' # <----- options: 'base', '_01'~'_06'
result_path = '/Users/lliu2/Documents/Chicago retrofits/ResStock results/'
iteration_path = f'cookcnty_2007{iteration}/baseline' 
baseline_result = os.path.join(result_path,iteration_path,'results_up00.parquet')
print(f'Result and file storage path: {iteration_path}')


Result and file storage path: cookcnty_2007_08/baseline


In [5]:
res = pd.read_parquet(baseline_result)
res

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_maximum_daily_use_overlap_kw,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw
0,1,34,2020-11-06 09:33:28,2020-11-06 09:34:45,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,5.884139,2.087214,1.554896,1.253012,15.090909,18.454545,11.031335,9.005224,3993.0,12.944411
1,2,5,2020-11-06 08:34:16,2020-11-06 08:35:49,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,21.361470,9.455010,6.586602,4.926889,15.363636,16.909091,49.236152,25.884132,4528.0,60.709783
2,3,12,2020-11-06 08:16:54,2020-11-06 08:18:31,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,7.862159,2.444959,2.588916,1.603292,15.454545,18.363636,18.737204,9.194733,3782.0,22.905679
3,4,19,2020-11-06 10:02:34,2020-11-06 10:20:02,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,116.325747,35.089375,25.945806,17.562567,15.000000,17.000000,329.141769,251.058302,5079.0,377.327134
4,5,19,2020-11-06 07:15:47,2020-11-06 07:17:13,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,4.866516,1.993636,1.467170,1.451682,17.818182,18.636364,8.884601,5.604362,5154.0,9.588941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,11,2020-11-06 07:38:40,2020-11-06 07:40:06,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,6.251742,1.150716,1.986324,1.160517,13.727273,17.545455,14.273349,6.799455,4524.0,15.932143
39996,39997,26,2020-11-06 09:30:32,2020-11-06 09:32:03,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,5.588132,2.010864,1.565217,1.486176,15.636364,18.818182,13.426244,5.668089,4528.0,17.594614
39997,39998,15,2020-11-06 06:43:41,2020-11-06 07:22:51,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,272.765407,62.355777,27.093392,34.431424,15.545455,16.909091,442.082491,380.838104,3326.0,606.100362
39998,39999,18,2020-11-06 09:06:28,2020-11-06 09:07:54,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,6.718392,2.613754,1.539309,1.143228,15.454545,15.727273,12.719581,9.030165,4503.0,13.992101


In [6]:
### HVAC cols changed after HVAC restructuring (implemented in Run5)
if iteration in ['base','_01','_02','_03','_04']:
    cooling_col = 'build_existing_model.hvac_system_cooling_type'
else:
    cooling_col = 'build_existing_model.hvac_cooling_type'
if iteration in ['base','_01','_02','_03','_04']:
    heating_col = 'build_existing_model.hvac_system_heating_natural_gas'
else:
    heating_col = 'build_existing_model.hvac_heating_type_and_fuel'
print('selected')


selected


### Check housing charateristics distributions

In [7]:
proto_chars = ['build_existing_model.geometry_stories',
              'build_existing_model.geometry_wall_type',
              'build_existing_model.vintage_acs',
              cooling_col,
              heating_col,
              'build_existing_model.geometry_floor_area'
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'>>> ResStock - {iteration_path} - RESULT SUMMARY:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['building_id'].count()
    Nchar = Nchar/Ns
    Nchar['N_failed'] = len(res[res[char].isnull()])
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(os.path.join(result_path, iteration_path, 'baseline_housing_char_breakdown.csv'), index=False)
print(f'>>> file saved to:\n  {os.path.join(result_path, iteration_path)}')

>>> ResStock - cookcnty_2007_08/baseline - RESULT SUMMARY:

  * 39961 / 40000 samples ran successfully, 39 failed, efficacy: 99.9% 

>>> Housing characteristics splits:

  * [1] build_existing_model.geometry_stories
1            0.544456
2            0.265809
3            0.189735
N_failed    39.000000
Name: building_id, dtype: float64

  * [2] build_existing_model.geometry_wall_type
Masonry      0.699357
WoodStud     0.300643
N_failed    39.000000
Name: building_id, dtype: float64

  * [3] build_existing_model.vintage_acs
1940-59      0.235104
1960-79      0.254999
1980-99      0.137909
2000-09      0.076074
2010s        0.011686
<1940        0.284227
N_failed    39.000000
Name: building_id, dtype: float64

  * [4] build_existing_model.hvac_cooling_type
Central AC     0.616927
Heat Pump      0.025600
None           0.052451
Room AC        0.305022
N_failed      39.000000
Name: building_id, dtype: float64

  * [5] build_existing_model.hvac_heating_type_and_fuel
Electricity ASHP        

### Add additional metrics to ResStock results

In [9]:
# (1) add sqft and eui
sqft_map = {'0-499': 328,
            '500-749': 633,
            '750-999': 885,
            '1000-1499': 1220,
            '1500-1999': 1690,
            '2000-2499': 2176,
            '2500-2999': 2663,
            '3000-3999': 3301,
            '4000+': 8194}

res['sqft'] = res['build_existing_model.geometry_floor_area'].map(sqft_map)
res['gas_eui'] = res['simulation_output_report.total_site_natural_gas_therm'].divide(res['sqft'])
res['elec_eui'] = res['simulation_output_report.total_site_electricity_kwh'].divide(res['sqft'])


# (2) aggregate heating and cooling end uses in ResStock results
res['total_gas_heating'] = res['simulation_output_report.natural_gas_central_system_heating_therm'] +\
    res['simulation_output_report.natural_gas_heating_therm']

res['total_elec_heating'] = res['simulation_output_report.electricity_central_system_heating_kwh'] +\
    res['simulation_output_report.electricity_central_system_pumps_heating_kwh'] +\
    res['simulation_output_report.electricity_fans_heating_kwh'] +\
    res['simulation_output_report.electricity_heating_kwh'] +\
    res['simulation_output_report.electricity_pumps_heating_kwh']

res['total_elec_cooling'] = res['simulation_output_report.electricity_central_system_cooling_kwh'] +\
    res['simulation_output_report.electricity_central_system_pumps_cooling_kwh'] +\
    res['simulation_output_report.electricity_cooling_kwh'] +\
    res['simulation_output_report.electricity_fans_cooling_kwh'] +\
    res['simulation_output_report.electricity_pumps_cooling_kwh']

print('new cols added to "res"')

### Get results by building prototypes from Elevate Energy
#### Note: cannot add prototype tags to ResStock results as prototypes overlap in chars

In [10]:
## Elevate Energy prototype chars
groups = pd.read_csv(os.path.join(result_path,'Groups.csv'))
print('prototypes loaded')


loaded


In [23]:
def get_res_by_prototype(res, row):
    res_group_i = res[res['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        res['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        res['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        res['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]
    
    return res_group_i

Metric_map1 = {'count': 'completed_status',
           'median gas': 'simulation_output_report.total_site_natural_gas_therm',
           'median elec': 'simulation_output_report.total_site_electricity_kwh'}
Metric_map2 = {'mean gas': 'simulation_output_report.total_site_natural_gas_therm',
           'mean elec': 'simulation_output_report.total_site_electricity_kwh'} # metric: res_var

## initialize
for metric in {**Metric_map1, **Metric_map2}:
    groups[metric] = np.NAN

### (1) get median consumption
for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    for metric, res_var in Metric_map1.items():
        if metric == 'count':
            groups.iloc[i, groups.columns.get_loc('count')] = \
                len(res_group_i[res_group_i['completed_status']=="Success"])
        else:
            groups.iloc[i, groups.columns.get_loc(metric)] = \
                res_group_i[res_var].median()

groups['Gas Diff-med(%)'] = ((groups['Non-normalized gas usage'] - groups['median gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-med(%)'] = ((groups['Non-normalized elec. usage'] - groups['median elec']) / groups['Non-normalized elec. usage']) * 100
print('>> median consumption computed')

### (2) get mean consumption
for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

groups['Gas Diff-mean(%)'] = ((groups['Non-normalized gas usage'] - groups['mean gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-mean(%)'] = ((groups['Non-normalized elec. usage'] - groups['mean elec']) / groups['Non-normalized elec. usage']) * 100
print('>> mean consumption computed')


>> median consumption computed
>> mean consumption computed


,HousingGroupNo,HousingGroupName,median gas,mean gas,median elec,mean elec
0,1,Masonry Post-1978 less than 2 stories,683.166282,731.543137,8768.758370,9946.311976
1,2,Masonry All Years Split Level,991.783098,1083.888463,8890.869095,10156.533266
2,3,Masonry Post-1978 2 or more stories,1074.116331,1243.332302,11035.517999,13109.331403
3,4,Masonry 1942-1978 Less than 2 Stories,887.116492,944.956665,8328.198301,9333.835069
4,5,Masonry 1942-1978 2 or more stories,1368.456184,1460.504704,9695.883175,11033.147404
5,6,Masonry Pre-1942 less than 2 stories,1033.165844,1117.046624,9051.373915,10152.824010
6,7,Masonry Pre-1942 2 or more stories,1423.328925,1484.019738,10212.558155,11780.647295
7,8,Frame Post-1978 Less than 2 stories,726.578844,777.614219,8486.764560,9665.518725
8,9,Frame Post-1978 Split Level,877.731546,1017.641463,9504.524754,11545.104939
9,10,Frame Post-1978 2 or more stories,1145.474099,1314.778638,11191.537518,13871.903045


In [24]:
### (3) get median & mean sqft & eui
Metric_map1 = {'median sqft': 'sqft',
               'median gas eui': 'gas_eui', 
               'median elec eui': 'elec_eui'}
Metric_map2 = {'mean sqft': 'sqft', 
               'mean gas eui': 'gas_eui', 
               'mean elec eui': 'elec_eui'}

## initialize
for metric in {**Metric_map1, **Metric_map2}:
    groups[metric] = np.NAN


for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    ### (3.1) get median values
    for metric, res_var in Metric_map1.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].median()

    ### (3.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

print('>> median & mean sqft & eui computed:')

groups[['HousingGroupNo','HousingGroupName','median gas','mean gas','median gas eui', 'mean gas eui',
        'median elec','mean elec', 'median elec eui','mean elec eui']]

>> median & mean sqft & eui computed


,HousingGroupNo,HousingGroupName,median gas,mean gas,median gas eui,mean gas eui,median elec,mean elec,median elec eui,mean elec eui
0,1,Masonry Post-1978 less than 2 stories,683.166282,731.543137,0.373161,0.409992,8768.758370,9946.311976,4.815646,5.588294
1,2,Masonry All Years Split Level,991.783098,1083.888463,0.717475,0.748961,8890.869095,10156.533266,6.277033,7.100540
2,3,Masonry Post-1978 2 or more stories,1074.116331,1243.332302,0.391818,0.420614,11035.517999,13109.331403,4.028665,4.418429
3,4,Masonry 1942-1978 Less than 2 Stories,887.116492,944.956665,0.717582,0.751510,8328.198301,9333.835069,6.625691,7.461365
4,5,Masonry 1942-1978 2 or more stories,1368.456184,1460.504704,0.897825,0.924193,9695.883175,11033.147404,6.239609,7.167381
5,6,Masonry Pre-1942 less than 2 stories,1033.165844,1117.046624,0.811437,0.868796,9051.373915,10152.824010,7.114303,8.068878
6,7,Masonry Pre-1942 2 or more stories,1423.328925,1484.019738,0.854202,0.870407,10212.558155,11780.647295,6.113600,7.083342
7,8,Frame Post-1978 Less than 2 stories,726.578844,777.614219,0.412344,0.446672,8486.764560,9665.518725,4.860369,5.558764
8,9,Frame Post-1978 Split Level,877.731546,1017.641463,0.404432,0.443523,9504.524754,11545.104939,4.513959,5.172212
9,10,Frame Post-1978 2 or more stories,1145.474099,1314.778638,0.394183,0.439624,11191.537518,13871.903045,3.945773,4.693686


In [28]:
### (4) get heating and cooling end uses to "groups"
Metric_map1 = ['frac_gas_heating', 'frac_elec_heating', 'frac_elec_cooling']
Metric_map2 = {'size_heating_kbut_h': 'simulation_output_report.size_heating_system_kbtu_h',
               'size_cooling_kbut_h': 'simulation_output_report.size_cooling_system_kbtu_h'}

## initialize
for metric in Metric_map1 + list(Metric_map2.keys()):
    groups[metric] = np.NAN

for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    ### (4.1) get fractions
    groups.iloc[i, groups.columns.get_loc('frac_gas_heating')] = \
        res_group_i['total_gas_heating'].mean()/res_group_i['simulation_output_report.total_site_natural_gas_therm'].mean()

    groups.iloc[i, groups.columns.get_loc('frac_elec_heating')] = \
        res_group_i['total_elec_heating'].mean()/res_group_i['simulation_output_report.total_site_electricity_kwh'].mean()
    
    groups.iloc[i, groups.columns.get_loc('frac_elec_cooling')] = \
        res_group_i['total_elec_cooling'].mean()/res_group_i['simulation_output_report.total_site_electricity_kwh'].mean()
    
    ### (4.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

print('computed')

computed


In [29]:
### (5) get heating & cooling and floor area char breakdowns

House = []
House_chars = [cooling_col, heating_col,
              'build_existing_model.geometry_floor_area',
              'build_existing_model.insulation_unfinished_attic']

for i, row in groups.iterrows():
    
    res_group_i = get_res_by_prototype(res, row)
    
    Ns = len(res_group_i.query('completed_status=="Success"'))
    Hi = []
    for char in House_chars:
        Nchar = res_group_i.groupby(char)['building_id'].count()/Ns # fractions
        Nchar = Nchar.rename('fraction').to_frame()
        Nchar['housing_char'] = Nchar.index.name
        Hi.append(Nchar)
    
    Hi = pd.concat(Hi, axis=0)
    Hi.index.name = 'sub_char'
    Hi = Hi.reset_index()
    Hi['HousingGroupNo'] = row['HousingGroupNo']
    
    House.append(Hi)
    
House = pd.concat(House, axis=0)
House = pd.pivot_table(House, values='fraction', index=['HousingGroupNo'],
                    columns=['housing_char','sub_char'], aggfunc=np.sum)

print('computed')

computed


In [30]:
## Combine "groups" + "House" into "Out" for export
out = groups.drop(['Stories','WallType','Vintage'], axis=1)
out = pd.concat([out.set_index('HousingGroupNo')], axis=1, keys=['none']).join(House)
out

none  \
                                     HousingGroupName   
HousingGroupNo                                          
1               Masonry Post-1978 less than 2 stories   
2                       Masonry All Years Split Level   
3                 Masonry Post-1978 2 or more stories   
4               Masonry 1942-1978 Less than 2 Stories   
5                 Masonry 1942-1978 2 or more stories   
6                Masonry Pre-1942 less than 2 stories   
7                  Masonry Pre-1942 2 or more stories   
8                 Frame Post-1978 Less than 2 stories   
9                         Frame Post-1978 Split Level   
10                  Frame Post-1978 2 or more stories   
11                Frame 1942-1978 less than 2 stories   
12                         Frame Pre-1942 Split Level   
13                  Frame 1942-1978 2 or more stories   
14                 Frame Pre-1942 less than 2 stories   
15                   Frame Pre-1942 2 or more stories   

                                                                            \
               Non-normalized gas usage Non-normalized elec. usage   count   
HousingGroupNo                                                               
1                                  1133                      10282   693.0   
2                                  1416                       9661  8959.0   
3                                  1812                      18225   553.0   
4                                  1210                       8779  4889.0   
5                                  1427                      10549  1157.0   
6                                  1504                       9299   729.0   
7                                  2044                      13272   938.0   
8                                  1054                       9353   718.0   
9                                  1168                      10445  1298.0   
10                                 1422                      13495   580.0   
11                                 1176                       9213  3617.0   
12                                 1297                       9964  2102.0   
13                                 1530                      12543   765.0   
14                                 1340                       9103   941.0   
15                                 1753                      11967  1161.0   

                                                                            \
                 median gas   median elec Gas Diff-med(%) Elec Diff-med(%)   
HousingGroupNo                                                               
1                683.166282   8768.758370       39.702888        14.717386   
2                991.783098   8890.869095       29.958821         7.971544   
3               1074.116331  11035.517999       40.722057        39.448461   
4                887.116492   8328.198301       26.684587         5.135001   
5               1368.456184   9695.883175        4.102580         8.087182   
6               1033.165844   9051.373915       31.305462         2.662932   
7               1423.328925  10212.558155       30.365512        23.051852   
8                726.578844   8486.764560       31.064626         9.261579   
9                877.731546   9504.524754       24.851751         9.004071   
10              1145.474099  11191.537518       19.446266        17.069007   
11               899.590238   8346.422769       23.504231         9.406027   
12              1236.170505   9638.742122        4.690015         3.264330   
13              1371.414360  10092.546205       10.365075        19.536425   
14               990.974624   8770.074504       26.046670         3.657316   
15              1432.641380  10327.374178       18.274879        13.701227   

                                           ...  \
                   mean gas     mean elec  ...   
HousingGroupNo                             ...   
1                731.543137   9946.311976  ...   
2           

### Write output to folder

In [31]:
#out_name = "Groups_results" + "_" + baseline_result.split("_")[2]
out.to_csv(os.path.join(result_path, iteration_path, 'baseline_prototype_results.csv'), index=True)
print(f'file saved to:\n  {os.path.join(result_path, iteration_path)}')

file saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_2007_08/baseline
